[the xlsxwriter format class](http://xlsxwriter.readthedocs.io/format.html)

In [13]:
import re
import string
import pandas as pd
import numpy as np
from xlsxwriter.utility import xl_col_to_name

In [14]:
def randStr(length):
    rand_str = "".join(np.random.choice(np.array(list(string.ascii_uppercase)),length))
    return rand_str

In [15]:
num = 7
column_names = [randStr(5) for i in range(num)]

my_df = pd.DataFrame(np.random.rand(num,num))
my_df.columns = column_names

In [16]:
my_df

,DNQJO,ZUBSO,XPCBL,UDIMS,TKKFW,QTYNZ,STGSM
0,0.738550,0.806423,0.721609,0.009137,0.232466,0.344570,0.503334
1,0.823926,0.272925,0.745531,0.438185,0.074280,0.787225,0.613182
2,0.739322,0.613388,0.840418,0.219731,0.164606,0.913096,0.325501
3,0.834222,0.881382,0.750082,0.717050,0.398981,0.253958,0.661224
4,0.419692,0.729984,0.371482,0.882860,0.505222,0.230476,0.675753
5,0.194444,0.416651,0.937145,0.468166,0.038413,0.991290,0.399311
6,0.216236,0.580901,0.338051,0.248347,0.452993,0.972451,0.732663


In [17]:
def spanXL(dataframe, start_index=0, row = 1):
    first = chr(65+start_index)
    last = xl_col_to_name(dataframe.shape[1]-1)
    span = [first+str(row),last+str(dataframe.shape[1]+1)]
    span = ":".join(span)
    return span
spanXL(my_df)

'A1:G8'

In [18]:
def colSpanXL(dataframe, start_index=0, row = 1):
    span_str = spanXL(dataframe, start_index, row).replace(":","&")
#     span_str = re.sub("\d+$","1",span_str)
    rx = re.compile("(\d.+\D)")
    col_span = "$".join(rx.split(span_str))
    return col_span

colSpanXL(my_df)

'A$1&G$8'

In [19]:
def makeSearch(term,df):
    search_str = '=SEARCH("{0}",{1})'.format(term,colSpanXL(df))
    return search_str

In [20]:
makeSearch("T",my_df)

'=SEARCH("T",A$1&G$8)'

In [21]:
def criteriaXL(term,df,xl_fun="SEARCH"):
    
    xl_fun = xl_fun.upper()
    
    if not xl_fun.startswith("="):
        xl_fun = "="+xl_fun
        
    
    fun_str = xl_fun+'("{0}",{1})'.format(term,colSpanXL(df))
    return fun_str

In [22]:
criteriaXL("acetyl",my_df)

'=SEARCH("acetyl",A$1&G$8)'

In [23]:
def dynFormat(format_set,workbook):
    return workbook.add_format(format_set)

In [26]:

# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('pandas_conditional.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
my_df.to_excel(writer, sheet_name='Sheet1',index=False)

# Get the xlsxwriter workbook and worksheet objects.
workbook  = writer.book
worksheet = writer.sheets['Sheet1']


worksheet.conditional_format(spanXL(my_df), {'type':'formula',
                                             'criteria': makeSearch(column_names[3],my_df),
                                             'format': dynFormat({"bg_color":"PeachPuff"},workbook)})

worksheet.conditional_format(spanXL(my_df),{'type':'formula',
                                             'criteria': makeSearch(column_names[-1],my_df),
                                             'format':  dynFormat({"bg_color":"orange"},workbook)})

# Close the Pandas Excel writer and output the Excel file.
writer.save()